In [1]:
import pandas as pd
from ast import literal_eval

metadata = pd.read_csv('./data/michgov6122023.csv').dropna(subset='Body').reset_index(drop=True)
for col in ['To', 'Cc']:
    metadata[col] = metadata[col].apply(literal_eval)
metadata

,Unnamed: 0,thread_index,parsed_email,image_lookup,From,Sent,To,Cc,Subject,Attachment,Body
0,1,0,From: Julie Skubik <julie@mhealthfund.com> Sen...,Staff_4_0000.txt,Julie Skubik <julie@mhealthfund.com,"Wednesday, June 03, 2015 3.34 PM","[;, Wendy (GOV)]",[],RE: Flint Water Situation MHEF Meeting,NaN,\nThank you. Do you need meto send an invite?\n
1,2,0,From: Wisniewski; Wendy (GOV) [mailto:wisniews...,Staff_4_0000.txt,Wisniewski; Wendy (GOV),"Wednesday, June 3,2015 3.33 PM",[Julie Skubik],[],RE: Flint Water Situation MHEF Meeting,NaN,\nAbsclutely. Here is all of my contact inforr...
2,3,0,From: Julie Skubik [mailtotiule@mhealthfundcom...,Staff_4_0000.txt,Julie Skubik,"Wednesday, June 03,2015 3.30 PM","[Wisniewski, Wendy (GOV)]",[],FW: Flint Water Situation MHEF Meeting,NaN,"\nHi Wendy- June 10""* at 11:0O am w} Paul Hill..."
3,4,0,"From: Hovey, Amy [mailto Amx Hovex@mailhousego...",Staff_4_0000.txt,"Hovey, Amy","Wednesday, June 3,2015 1.56 PM",[Julie Skubik],[],RE: Flint Water Situation MHEF Meeting,NaN,"\nHi Julie\nJune 10t at 11AMwculd be great, Wh..."
4,5,0,From: Julie Skubik [mailto iulie@mhezlthfundco...,Staff_4_0000.txt,Julie Skubik,"Wednesday, June 03, 2015 10.59 AM","[Hovey, Amy Subject:]",[],Flint Water Situation MHEF Meeting,NaN,"\nXi Arny"" would Ilke to set up & meeting with..."
...,...,...,...,...,...,...,...,...,...,...,...
26154,32176,16920,"From: Stoken, Laura (GOV) Sent: MondayDecember...",Staff_2_0920.txt,"Stoken, Laura (GOV)",NaN,[],[],Executive Office of the Governor\n,NaN,Dear Thank you for your recent correspondence ...
26155,32178,16921,"From: Stoken, Laura (GOV) Sent: Mondav_Decembe...",Staff_2_0922.txt,"Stoken, Laura (GOV)",NaN,[],[],Executive Office of the Governor\n,NaN,Dear Thank you for your recent correspondence ...
26156,32180,16922,From: Stoken; Laura (GOV) Sent: Monday Decembe...,Staff_2_0924.txt,Stoken; Laura (GOV),"Monday December 28, 2015 10.50 AM",[],[],Executive Office of the Governor\nDearl,NaN,Thank you for your recent correspondence sent...
26157,32182,16923,"From: Stoken, Laura (GOV) Sent: MondayDecember...",Staff_2_0926.txt,"Stoken, Laura (GOV)",NaN,[],[],Executive Office of the Governor\n,NaN,Dear Thank you for your recent correspondence ...


In [2]:
input_texts = list(metadata['Body'])

In [4]:
from txtai.embeddings import Embeddings
embeddings = Embeddings()
embeddings.load('./data/fl_index6122023')

In [25]:
embeddings.search('query: contaminated water', limit=5)

[(2505, 0.8385169506072998),
 (16264, 0.8384946584701538),
 (18856, 0.8383601903915405),
 (15122, 0.8372467756271362),
 (14082, 0.8372388482093811)]

In [29]:
input_texts[14082]

'Redacted\nOUMI Update (Hollins)\n*Flint (waiting for report from DEQ on water situation)'

In [30]:
metadata.iloc[2505]

Unnamed: 0                                                   3202
thread_index                                                 1374
parsed_email    From: Murray, David (GOV) Sent: Thursday; Sept...
image_lookup                                     Staff_9_0860.txt
From                                          Murray, David (GOV)
Sent                                  Thursday; September 24,2015
To                                        [Ackerman, Darin (GOV)]
Cc                                                             []
Subject                                                  FW: Data
Attachment                                                    NaN
Body             Attachments: Pediatric Lead Exposure Flint Wa...
Name: 2505, dtype: object

In [35]:
dit_logits = pd.read_csv('./data/dit_logits_embedded.csv')
dit_logits.aws_path.loc[dit_logits.aws_path.str.contains('Staff_9_0860')].iloc[0]

'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_9_jp2/Staff_9_0860.png'

In [44]:
from sentence_transformers import CrossEncoder
def retrieval_rerank(query, model_path, rerank_model_path, input_texts, limit):
    embeddings = Embeddings()
    embeddings.load(model_path)
    s = embeddings.search(query, limit=limit)
    rerank_model = CrossEncoder(rerank_model_path, max_length=512)
    scores = rerank_model.predict([(query, input_texts[tup[0]]) for tup in s])
    reranked = [s[score][0] for score in scores.argsort()[::-1]]
    return reranked

In [45]:
reranked = retrieval_rerank(
    'neglience of government officials',
    './data/fl_index622023',
    'cross-encoder/ms-marco-MiniLM-L-2-v2',
    input_texts,
    10
)
reranked

[11802, 4595, 17313, 16927, 4863, 6970, 20752, 5833, 16166, 7922]

In [48]:
def lookup_image(rerank_id):
    return dit_logits.aws_path.loc[dit_logits.aws_path.str.contains(metadata.iloc[rerank_id].image_lookup.split('.')[0])].iloc[0]

In [49]:
[lookup_image(rr) for rr in reranked]

['https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_5_jp2/Staff_5_0478.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_18_jp2/Staff_18_0031.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_17_jp2/Staff_17_0321.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_17_jp2/Staff_17_0046.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_18_jp2/Staff_18_0230.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_12_jp2/Staff_12_0323.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_15_jp2/Staff_15_0097.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_13_jp2/Staff_13_0708.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_14_jp2/Staff_14_1688.png',
 'https://rjdgrlmrpwnzwmdwgizseanca.s3.us-east-2.amazonaws.com/Staff_12_jp2/Staff_12_1171.png']

In [64]:
thread_id = metadata.iloc[2505].thread_index
metadata.iloc[metadata.groupby('thread_index').groups[thread_id]].image_lookup.unique()[0]

'Staff_9_0860.txt'